<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/fitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
def fit_naive(Model, Batcher, loss, initial_epoch, max_epoch):
    X_train, X_test, Y_train, Y_test = Batcher.load_data()
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)  

    Model.fit(x=X_train, y=Y_train, batch_size=100, epochs = max_epoch, initial_epoch=initial_epoch, shuffle=True, validation_data=(X_test, Y_test))
    # callbacks=[early_stopping, reduce_lr, checkpoint]

In [ ]:
def fit_simMat(Model, Batcher, loss, initial_epoch, max_epoch):
  X, Y = Batcher.load_data()
  X = np.reshape(X,(-1,128,128,1))
  Y = np.reshape(Y,(-1))
  print(X.shape)
  print(Y.shape)

  # x를 batch 생성 함수로 한다면, batch_size를 사용하지 않고(사이즈는 어짜피 batch 생성 함수에서 결정) steps_per_epoch으로 한 에폭에서 batch 개수를 설정
  # 어떻게 배치까지 다 모여있는 X를 fit에 배치별로 처리하라 입력하지? generator 함수의 형태로 한 배치씩 내놓도록 해야하나.
  Model.fit(x=X, y=Y, batch_size=25, epochs=50, shuffle=False)

In [ ]:
def fit(fit_name, Model, Batcher, loss, initial_epoch, max_epoch):
    if fit_name == 'naive_fit': fit_naive(Model, Batcher, loss, initial_epoch, max_epoch)
    elif fit_name == 'simMat_fit': fit_simMat(Model, Batcher, loss, initial_epoch, max_epoch)
        